In [1]:
import requests
import numpy as np
import pandas as pd
import bs4 as BeautifulSoup
import re

In [73]:
response = requests.get('https://api.fbi.gov/wanted/v1/list', params={
    'page': 27,
    'pageSize': 20
}).json()['items']

### Clean Details

In [74]:
fugitive = pd.DataFrame(response)
details = re.sub(r'[\n\xa0]', '', BeautifulSoup.BeautifulSoup(fugitive.details[0]).get_text())

TypeError: object of type 'NoneType' has no len()

In [55]:
details

"The FBI's Albuquerque Field Office, Farmington Resident Agency in New Mexico is seeking the public's assistance in finding the person(s) responsible for the death of Isiah Terrell Billy, who lived in Shiprock, New Mexico, on the Navajo Nation.On October 5, 2020, Billy was found deceased in a wash east of the Sinclair gas station near mile marker 23 on U.S. Highway 64, in Shiprock, New Mexico. The cause of death was ruled to be strangulation."

In [23]:
fugitive.dtypes

languages                  object
reward_max                  int64
weight_max                float64
publication                object
age_range                  object
reward_min                  int64
place_of_birth             object
possible_countries         object
sex                        object
race                       object
reward_text                object
height_min                float64
height_max                float64
possible_states            object
race_raw                   object
aliases                    object
title                      object
caution                    object
build                      object
url                        object
subjects                   object
scars_and_marks            object
legat_names                object
person_classification      object
suspects                   object
nationality                object
eyes_raw                   object
age_min                   float64
details                    object
hair          

### Clean Remarks

In [24]:
remarks = BeautifulSoup.BeautifulSoup(fugitive.remarks[0]).get_text()

TypeError: object of type 'NoneType' has no len()

In [25]:
if fugitive.remarks[0] is not None:
    remarks = BeautifulSoup.BeautifulSoup(fugitive.remarks[0]).get_text()
    print(1)
else:
    print(0)


0


### Clean NCIC

In [26]:
fugitive.ncic # Just a str, assignment

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
Name: ncic, dtype: object

### Scars and Marks

In [27]:
if fugitive.scars_and_marks[0] is not None:
    re.sub(r'[\n\r]', '', fugitive.scars_and_marks[0])
    print(1)
else:
    print(2)

2


In [28]:
fugitive.images

0     [{'large': 'https://www.fbi.gov/wanted/seeking...
1     [{'large': 'https://www.fbi.gov/wanted/counter...
2     [{'large': 'https://www.fbi.gov/wanted/vicap/m...
3     [{'large': 'https://www.fbi.gov/wanted/seeking...
4     [{'large': 'https://www.fbi.gov/wanted/kidnap/...
5     [{'large': 'https://www.fbi.gov/wanted/kidnap/...
6     [{'large': 'https://www.fbi.gov/wanted/additio...
7     [{'large': 'https://www.fbi.gov/wanted/vicap/m...
8     [{'large': 'https://www.fbi.gov/wanted/cei/ope...
9     [{'large': 'https://www.fbi.gov/wanted/vicap/m...
10    [{'large': 'https://www.fbi.gov/wanted/vicap/u...
11    [{'large': 'https://www.fbi.gov/wanted/seeking...
12    [{'large': 'https://www.fbi.gov/wanted/counter...
13    [{'large': 'https://www.fbi.gov/wanted/wcc/fre...
14    [{'large': 'https://www.fbi.gov/wanted/vicap/m...
15    [{'large': 'https://www.fbi.gov/wanted/kidnap/...
16    [{'large': 'https://www.fbi.gov/wanted/vicap/m...
17    [{'large': 'https://www.fbi.gov/wanted/cyb

In [29]:
images = pd.DataFrame(fugitive.images[0])
images.original

0    https://www.fbi.gov/wanted/seeking-info/attemp...
1    https://www.fbi.gov/wanted/seeking-info/attemp...
2    https://www.fbi.gov/wanted/seeking-info/attemp...
Name: original, dtype: object

In [30]:
class Images:
    def __init__(self, df):
        self.df = pd.DataFrame(df)
        self.original = self.df.original.tolist()
        self.large = self.df.large.tolist()
        self.caption = self.df.caption.tolist()
        self.thumb = self.df.thumb.tolist()

In [31]:
imageClass = Images(df=fugitive.images[0])

In [32]:
imageClass.large

['https://www.fbi.gov/wanted/seeking-info/attempted-armored-car-robbery/@@images/image/large',
 'https://www.fbi.gov/wanted/seeking-info/attempted-armored-car-robbery/pharmoredcar3.png/@@images/image/large',
 'https://www.fbi.gov/wanted/seeking-info/attempted-armored-car-robbery/pharmoredcar1.png/@@images/image/large']

In [33]:
fugitive.person_classification

0       Main
1       Main
2       Main
3       Main
4       Main
5       Main
6       Main
7       Main
8       Main
9       Main
10      Main
11      Main
12      Main
13      Main
14      Main
15      Main
16      Main
17      Main
18      Main
19    Victim
Name: person_classification, dtype: object

In [34]:
fugitive.person_classification[0]

'Main'

In [35]:
fugitive.weight

0                                                  None
1                                                  None
2                  60 pounds (at time of disappearance)
3                                            140 pounds
4                                            115 pounds
5     Approximately 40 to 42 pounds (at time of disa...
6                                     196 to 201 pounds
7                                            190 pounds
8                                                  None
9                              Approximately 170 pounds
10                                    114 to 145 pounds
11                                                 None
12                                                 None
13                                    209 to 210 pounds
14                                           160 pounds
15                                           115 pounds
16                                           160 pounds
17                                              

# Weight
What we can do is split the string into a list, and check for all elements in it:
Condtions:
1 int: weight (lbs)
2 int: (x + y ) /2 to get average
1 int + kg str: convert to lbs
2 int + kg str:

In [36]:
fugitive.height_min

0      NaN
1      NaN
2     54.0
3     66.0
4     66.0
5     40.0
6     73.0
7     68.0
8      NaN
9     70.0
10    62.0
11     NaN
12     NaN
13    68.0
14    71.0
15    66.0
16    72.0
17     NaN
18    69.0
19    64.0
Name: height_min, dtype: float64

In [66]:
fugitive.race[1]

'white'

In [67]:
fugitive.age_range

0                                        None
1                                        None
2      6 years old (at time of disappearance)
3                                        None
4                                        None
5                                        None
6                                        None
7     36 years old (at time of disappearance)
8                                        None
9                  Approximately 22 years old
10                         20 to 45 years old
11                                       None
12                                       None
13                                       None
14              22 (at time of disappearance)
15                                       None
16                            Forty years old
17                                       None
18                                       None
19                                       None
Name: age_range, dtype: object

In [68]:
fugitive.aliases

0                                                  None
1         [Boris Levitan, Boris Livshitc, David Wetzky]
2                                                  None
3                                             ["Molly"]
4                              [Susan Gail Carter Webb]
5        [Aganza Maria Ochoa Lopez, Aranza Ochoa Lopez]
6                                                  None
7                                                  None
8                                                  None
9                                                  None
10                                                 None
11                                                 None
12                                [Aleksandr Korshunov]
13                     [Fred Arias, Frederick M. Arias]
14                                                 None
15                     [Mary E. Davis, Mary E. Johnson]
16                                                 None
17    [Yevhgyeniy Polyanin, Yevgeniy Polyanin, Y

In [178]:
for alias in fugitive.aliases[0]:
    if '"' in alias or '“' in alias:
        match = re.search(r'"(.*?)"|“(.*?)”|“(.*?)"|"(.*?)“', alias).group(0)
        match = re.sub(r'[\"|\“]', "", match)
        print(f"Alias: {alias}, Match: {match}")
    else:
        print(alias)


TypeError: 'NoneType' object is not iterable

[“Mystickal”, "Mystikal", “Rizzus", "Russel", "Crusty Rusty"]

In [186]:
fugitive.uid

0     712c3469993e4eb8ad706b65082b37ef
1     d1af1b9c5c3648ebb1cf926cb8c4fb09
2     64cd2584930e49278a2951b71e12717e
3     952a0f45262b4a3a80f3d7839dda45fd
4     66802fe008634da18d76070e809e2271
5     32eeba50d906400786c23ed38133fd04
6     35864dde9dbe4d058f9e372a6ea04b4f
7     7e2490248a2746ccb011d7d11abea5a9
8     ed0db933b9154f6c8b67b67db42a7f77
9     421f5461e4b34c82a6930bd1ebb0d792
10    1e8ab5f701bd4497bf3344a9935f2a7a
11    6bc39f5fe60a404899acd3ea41999ec1
12    c9444d9fc13f47de89f25b1a17249021
13    1ef7b9c988984e4aa6d87a8557fbbf1f
14    ba916f70bdcd4ee8872b6e6f76643be1
15    b5e7d23089834cf79709b9042400ca8e
16    9957ba299d774955b857bad921685cac
17    5a57f941293242e5b02b1d6069f19f8c
18    d116fc824786453eaa1f2aca9b34ca8e
19    f80e60096f844abb979f2688b039f7ec
Name: uid, dtype: object

In [212]:
fugitive.locations

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
Name: locations, dtype: object

In [223]:
status = None if fugitive.status[9] == "na" else None
print(status)

None


In [233]:
fugitive.reward_max

0         0
1         0
2         0
3         0
4         0
5         0
6         0
7     10000
8         0
9         0
10        0
11        0
12        0
13        0
14        0
15        0
16        0
17        0
18        0
19        0
Name: reward_max, dtype: int64

fugitive.reward_max[7]

In [235]:
type(fugitive.reward_max[7])

numpy.int64

In [264]:
fugitive.reward_min

0           0
1     1000000
2     1000000
3           0
4           0
5           0
6           0
7           0
8           0
9           0
10          0
11          0
12          0
13          0
14          0
15          0
16          0
17          0
18          0
19          0
Name: reward_min, dtype: int64

In [266]:
type(fugitive.reward_min[1])

numpy.int64

In [304]:
fugitive.files

0     [{'url': 'https://www.fbi.gov/wanted/additiona...
1     [{'url': 'https://www.fbi.gov/wanted/cei/semio...
2     [{'url': 'https://www.fbi.gov/wanted/counterin...
3     [{'url': 'https://www.fbi.gov/wanted/seeking-i...
4     [{'url': 'https://www.fbi.gov/wanted/vicap/uni...
5     [{'url': 'https://www.fbi.gov/wanted/seeking-i...
6     [{'url': 'https://www.fbi.gov/wanted/seeking-i...
7     [{'url': 'https://www.fbi.gov/wanted/wcc/mykal...
8     [{'url': 'https://www.fbi.gov/wanted/murders/m...
9     [{'url': 'https://www.fbi.gov/wanted/seeking-i...
10    [{'url': 'https://www.fbi.gov/wanted/kidnap/na...
11    [{'url': 'https://www.fbi.gov/wanted/seeking-i...
12    [{'url': 'https://www.fbi.gov/wanted/vicap/uni...
13    [{'url': 'https://www.fbi.gov/wanted/additiona...
14    [{'url': 'https://www.fbi.gov/wanted/vicap/uni...
15    [{'url': 'https://www.fbi.gov/wanted/seeking-i...
16    [{'url': 'https://www.fbi.gov/wanted/seeking-i...
17    [{'url': 'https://www.fbi.gov/wanted/vicap

In [306]:
fugitive

,languages,reward_max,weight_max,publication,age_range,reward_min,place_of_birth,possible_countries,sex,race,...,description,locations,field_offices,weight,dates_of_birth_used,occupations,weight_min,age_max,modified,@id
0,None,0,160.0,2022-11-03T11:41:00,None,0,Haiti,None,Male,black,...,Conspiracy to Commit Hostage Taking; Hostage T...,None,[miami],160 pounds,None,None,160.0,NaN,2023-01-16T00:13:23+00:00,https://api.fbi.gov/@wanted-person/b54a9ec0a2f...
1,None,0,290.0,2010-07-20T16:25:00,None,0,"Kyiv, Ukraine",None,Male,white,...,Fraud by Wire; RICO Conspiracy; Mail Fraud; Mo...,None,[philadelphia],290 pounds,"[June 30, 1946, July 5, 1946]",[Businessman],290.0,NaN,2023-01-16T00:13:22+00:00,https://api.fbi.gov/@wanted-person/34c738a021b...
2,None,0,NaN,2020-06-04T10:57:00,None,0,Russia,None,Male,white,...,Conspiracy to Defraud the United States,None,[washingtondc],None,"[December 21, 1958]",None,NaN,NaN,2023-01-16T00:13:22+00:00,https://api.fbi.gov/@wanted-person/e8333eccb19...
3,None,0,NaN,2022-10-05T12:41:00,None,0,None,None,None,None,...,"Murder Victims\r\nFitzgerald, Georgia\r\nSepte...",None,[atlanta],None,None,None,NaN,NaN,2023-01-16T00:13:21+00:00,https://api.fbi.gov/@wanted-person/38f36a4016a...
4,None,0,200.0,2023-01-12T11:08:00,20 to 50 years old,0,None,None,Female,white,...,"December 2, 2022\r\nRiceboro, Georgia",None,None,185 to 200 pounds,None,None,185.0,50.0,2023-01-16T00:13:21+00:00,https://api.fbi.gov/@wanted-person/f481dd60ff6...
5,None,0,NaN,2022-02-22T12:11:00,None,0,None,None,None,None,...,"Homicide Victims\r\nConway, South Carolina\r\n...",None,[columbia],None,None,None,NaN,NaN,2023-01-16T00:13:21+00:00,https://api.fbi.gov/@wanted-person/c7acc640dab...
6,None,0,NaN,2020-08-04T14:04:00,None,0,None,None,None,None,...,"Salt Lake City, Utah\r\nMay 30, 2020",None,[saltlakecity],None,None,None,NaN,NaN,2023-01-16T00:13:20+00:00,https://api.fbi.gov/@wanted-person/90aa56f0240...
7,None,0,290.0,2020-09-29T07:47:00,None,0,Illinois,None,Male,white,...,Securities Fraud (One Count); Wire Fraud (Six ...,None,[lasvegas],290 pounds,"[October 4, 1969, October 4, 1966]",None,290.0,NaN,2023-01-16T00:13:19+00:00,https://api.fbi.gov/@wanted-person/fe16684709a...
8,None,0,160.0,2020-09-25T07:32:00,None,0,Mexico,None,Male,hispanic,...,Unlawful Flight to Avoid Prosecution - Murder,None,[sacramento],160 pounds,"[October 21, 1975]",[Farm Laborer],160.0,NaN,2023-01-16T00:13:18+00:00,https://api.fbi.gov/@wanted-person/dfcf24ee89c...
9,None,0,NaN,2020-09-28T12:48:00,None,0,None,None,None,None,...,Homicide Victim\r\nWarm Springs Indian Reserva...,None,[portland],None,None,None,NaN,NaN,2023-01-16T00:13:18+00:00,https://api.fbi.gov/@wanted-person/76ed29d2b65...


In [15]:
fugitive.subjects

0     [ViCAP Unidentified Persons]
1     [ViCAP Unidentified Persons]
2     [ViCAP Unidentified Persons]
3     [ViCAP Unidentified Persons]
4          [ViCAP Missing Persons]
5          [ViCAP Missing Persons]
6          [ViCAP Missing Persons]
7          [ViCAP Missing Persons]
8          [ViCAP Missing Persons]
9          [ViCAP Missing Persons]
10         [ViCAP Missing Persons]
11         [ViCAP Missing Persons]
12         [ViCAP Missing Persons]
13         [ViCAP Missing Persons]
14         [ViCAP Missing Persons]
15         [ViCAP Missing Persons]
16         [ViCAP Missing Persons]
17         [ViCAP Missing Persons]
18         [ViCAP Missing Persons]
19         [ViCAP Missing Persons]
Name: subjects, dtype: object

In [29]:
fugitive.possible_states[6]

['US-PA', 'US-WA']

In [30]:
states = []
for state in fugitive.possible_states[6]:
    states.append(state.replace("US-", ""))

In [31]:
states

['PA', 'WA']

In [38]:
fugitive.warning_message

0                                                  None
1                                                  None
2                                                  None
3                                                  None
4                                                  None
5                   SHOULD BE CONSIDERED AN ESCAPE RISK
6     SHOULD BE CONSIDERED AN INTERNATIONAL FLIGHT RISK
7     SHOULD BE CONSIDERED AN INTERNATIONAL FLIGHT RISK
8     SHOULD BE CONSIDERED AN INTERNATIONAL FLIGHT RISK
9     SHOULD BE CONSIDERED AN INTERNATIONAL FLIGHT RISK
10    SHOULD BE CONSIDERED AN INTERNATIONAL FLIGHT RISK
11    SHOULD BE CONSIDERED AN INTERNATIONAL FLIGHT RISK
12                                                 None
13                                                 None
14    SHOULD BE CONSIDERED AN INTERNATIONAL FLIGHT RISK
15                                                 None
16                                                 None
17                                              

In [42]:
fugitive.warning_message[6].lower()

'should be considered an international flight risk'

In [44]:
fugitive.title

0                        Wang, Lei
1                     Li, Shaoming
2                        Lin, Yong
3                         Ye, Jian
4                    Wang, Hongwei
5        CHRISTOPHER ROBERT METSOS
6                       LI FANGWEI
7                      NADER SAEDI
8                    HAMID FIROOZI
9                     AMIN SHOKOHI
10                    SINA KEISSAR
11    MOHAMMAD SADEGH AHMADZADEGAN
12                FARHAN UL ARSHAD
13                     AHMAD FATHI
14                OMID GHAFFARINIA
15                 NOOR AZIZ UDDIN
16                       WEN XINYU
17                    HUANG ZHENYU
18                       WANG DONG
19                      GU CHUNHUI
Name: title, dtype: object

In [51]:
print((''.join([c for c in fugitive.title[4] if c.isalpha()])).lower())

wanghongwei


In [56]:
fugitive.age_min

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
5      NaN
6      NaN
7      NaN
8      NaN
9      NaN
10     NaN
11     NaN
12     NaN
13     NaN
14    26.0
15     NaN
16     NaN
17     NaN
18     NaN
19     NaN
Name: age_min, dtype: float64

In [57]:
26.0

26.0

In [75]:
fugitive.reward_text

0                                                  None
1                                                  None
2     The FBI is offering a reward of up to $10,000 ...
3                                                  None
4                                                  None
5                                                  None
6                                                  None
7     The FBI is offering a reward of up to $10,000 ...
8     The United States Government is offering a rew...
9     The FBI is offering a reward of up to $25,000 ...
10    The Rewards for Justice Program, United States...
11    The Rewards For Justice Program, United States...
12    The FBI is offering a reward of up to $20,000 ...
13                                                 None
14                                                 None
15    The FBI is offering a reward of up to $10,000 ...
16                                                 None
17                                              

In [84]:
reward = re.findall(r'(?:[\£\$\€]{1}[,\d]+.?\d*)', fugitive.reward_text[9])

In [85]:
reward[0].strip(" ")

'$25,000'

In [131]:
reward2 = fugitive.reward_text[8].split(" ")

In [132]:
reward2

['The',
 'United',
 'States',
 'Government',
 'is',
 'offering',
 'a',
 'reward',
 'of',
 'up',
 'to',
 '$1',
 'million',
 'dollars',
 'for',
 'information',
 'directly',
 'leading',
 'to',
 'the',
 'arrest',
 'or',
 'prosecution',
 'of',
 'those',
 'responsible',
 'for',
 'the',
 'kidnapping',
 'and',
 'captivity',
 'of',
 'Caitlan',
 'Coleman',
 'and',
 'her',
 'family.']

In [156]:
def get_reward(self):
    reward = self.reward_text[8].split(" ")
    for i, s in enumerate(reward):
        value = re.findall(r'(?:[\£\$\€][,\d]+.?\d*)', s)
        output = re.sub(r'[$,]', '', "".join(value))
        if output.__len__() < 2:
            if reward[i + 1] == "million":
                return int(output) * 1000000
        else:
            return output

val = get_reward(fugitive)
print(type(np.int32(val)))

<class 'numpy.int32'>
